## Bidirectional Lstm Rnn

In [2]:
import pandas as pd

In [4]:
df=pd.read_csv('news.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [25]:
df=df.dropna()

In [26]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [27]:
df.shape

(18285, 5)

In [28]:
# independent
X=df.drop('label',axis=1)

In [29]:
y=df['label']

In [30]:
X.shape

(18285, 4)

In [31]:
y.shape

(18285,)

In [32]:
y.value_counts() # check data whether balanced??

label
0    10361
1     7924
Name: count, dtype: int64

In [33]:
import tensorflow as tf

2025-03-08 10:25:11.608989: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 10:25:11.763014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741409711.822950    7329 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741409711.840662    7329 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 10:25:12.013002: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [38]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [54]:
#vocabulary size
voc_size=5000

In [39]:
messages=X.copy()

In [40]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [41]:
#considering tittle

In [42]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/rgukt-
[nltk_data]     basar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
messages.reset_index(inplace=True)

In [51]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()

    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

    

In [52]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [58]:
onehot_rep=[one_hot(words,voc_size) for words in corpus]
onehot_rep # can see the index of particular word that appears

[[1157, 663, 1346, 4973, 3279, 104, 1726, 2700, 4042, 2348],
 [153, 3415, 3434, 4072, 4723, 1100, 1055],
 [1188, 96, 2159, 2892],
 [2686, 2875, 1977, 737, 3384, 4317],
 [4678, 4723, 3994, 435, 2456, 3075, 4723, 3552, 776, 2866],
 [4330,
  1283,
  4506,
  631,
  97,
  2181,
  1567,
  2173,
  3852,
  2356,
  4522,
  863,
  3842,
  2927,
  1055],
 [1764, 1435, 3960, 3291, 3919, 1734, 2877, 1090, 3351, 652, 2814],
 [2379, 2507, 2011, 4768, 564, 1392, 2181, 1138, 3351, 652, 2814],
 [1788, 2629, 1203, 3014, 967, 174, 2645, 2065, 2181, 2188],
 [1431, 4484, 2949, 183, 2270, 552, 2843, 980],
 [1371, 925, 3763, 4913, 2839, 2247, 413, 223, 2302, 4973, 4417],
 [737, 1904, 3279, 174, 2181, 564],
 [4115, 87, 1705, 1118, 701, 2617, 2115, 4447, 61],
 [2644, 2643, 4651, 1127, 1243, 970, 4505, 3351, 652, 2814],
 [1909, 3232, 186, 1708, 2164, 3351, 652, 2814],
 [3774, 1723, 3266, 2769, 1431, 4368, 334, 4020, 2499, 1811],
 [1022, 312, 3415],
 [867, 1618, 3495, 3075, 2181, 3086, 375, 1055],
 [2975, 3075, 3

## Embedding Rep

In [59]:
sent_length=20 # mustbe not be less than length

In [62]:
embadded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)

In [64]:
embadded_docs # observe padding

array([[   0,    0,    0, ..., 2700, 4042, 2348],
       [   0,    0,    0, ..., 4723, 1100, 1055],
       [   0,    0,    0, ...,   96, 2159, 2892],
       ...,
       [   0,    0,    0, ..., 3351,  652, 2814],
       [   0,    0,    0, ..., 1732, 3162,  597],
       [   0,    0,    0, ..., 4686, 2175,  198]], dtype=int32)

### creating model

In [72]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))

model.add(Bidirectional(LSTM(100))) # 100 newrond farword & back
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [83]:
embadded_docs

array([[   0,    0,    0, ..., 2700, 4042, 2348],
       [   0,    0,    0, ..., 4723, 1100, 1055],
       [   0,    0,    0, ...,   96, 2159, 2892],
       ...,
       [   0,    0,    0, ..., 3351,  652, 2814],
       [   0,    0,    0, ..., 1732, 3162,  597],
       [   0,    0,    0, ..., 4686, 2175,  198]], dtype=int32)

In [84]:
import numpy as np
X_final=np.array(embadded_docs)
y_final=np.array(y)

In [85]:
X_final

array([[   0,    0,    0, ..., 2700, 4042, 2348],
       [   0,    0,    0, ..., 4723, 1100, 1055],
       [   0,    0,    0, ...,   96, 2159, 2892],
       ...,
       [   0,    0,    0, ..., 3351,  652, 2814],
       [   0,    0,    0, ..., 1732, 3162,  597],
       [   0,    0,    0, ..., 4686, 2175,  198]], dtype=int32)

In [86]:
y_final

array([1, 0, 1, ..., 0, 1, 1])

In [91]:
#train-test-split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [94]:
#model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.8345 - loss: 0.3582 - val_accuracy: 0.9135 - val_loss: 0.2102
Epoch 2/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9482 - loss: 0.1303 - val_accuracy: 0.9167 - val_loss: 0.2136
Epoch 3/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9701 - loss: 0.0875 - val_accuracy: 0.9142 - val_loss: 0.2218
Epoch 4/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.9833 - loss: 0.0555 - val_accuracy: 0.9138 - val_loss: 0.2837
Epoch 5/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9917 - loss: 0.0288 - val_accuracy: 0.9099 - val_loss: 0.3806
Epoch 6/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9946 - loss: 0.0197 - val_accuracy: 0.9120 - val_loss: 0.3878
Epoch 7/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9982 - loss: 0.0081 - val_accuracy: 0.9089 - val_loss: 0.4837
Epoch 8/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.9986 - loss: 0.0036 - val_acc

In [95]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step    


In [96]:
y_pred

array([[9.9939138e-01],
       [1.7785352e-08],
       [1.9724441e-05],
       ...,
       [8.4274925e-06],
       [9.9730092e-01],
       [9.9253827e-01]], dtype=float32)

In [97]:
y_pred>=0.5

array([[ True],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [ True]])

In [99]:
y_pred=np.where(y_pred>=0.5,1,0)

In [104]:
from sklearn.metrics import confusion_matrix,accuracy_score

confusion_matrix(y_test,y_pred)

array([[3184,  235],
       [ 353, 2263]])

In [105]:
print(accuracy_score(y_test,y_pred))

0.9025683512841757
